# Introduction

In [ ]:
using PyCall

py"""
def pymysum(a):
    result = 0
    for x in a:
        result += x
    return result
"""

pymysum = py"pymysum"

In [ ]:
N = 10_000_000
a = float.(collect(1:N))

pymysum(a)
@time pymysum(a)

In [ ]:
@pyimport numpy as np

In [ ]:
np.sum(a)
@time np.sum(a)

python's `main()` can be found here: https://github.com/python/cpython/blob/master/Programs/python.c

```C
/* Minimal main program -- everything is loaded from the library */

#include "Python.h"

#ifdef MS_WINDOWS
int
wmain(int argc, wchar_t **argv)
{
    return Py_Main(argc, argv);
}
#else
int
main(int argc, char **argv)
{
    return _Py_UnixMain(argc, argv);
}
#endif
```

In [ ]:
# first call compiles the function
sum(a)

In [ ]:
@time sum(a)

In [ ]:
function mysum(a)
    result = zero(eltype(a))
    for x in a
        result += x
    end
    result
end

In [ ]:
mysum(a)
@time mysum(a)

In [ ]:
@less sum(a)

In Julia `sum()` is defined as:
```julia
sum(a) = mapreduce(identity, +, a)
sum(a::AbstractArray{Bool}) = countnz(a)
```

Doing `julia> @less mapreduce(identity, +, a)` points to the same file, where we find:

```julia
## reduce & mapreduce

# `mapreduce_impl()` is called by `mapreduce()` (via `_mapreduce()`, when `A`
# supports linear indexing) and does actual calculations (for `A[ifirst:ilast]` subset).
```

The actual code:

```julia
function mapreduce_impl(f, op, A::AbstractArray, ifirst::Integer, ilast::Integer, blksize::Int=pairwise_blocksize(f, op))
    if ifirst == ilast
        @inbounds a1 = A[ifirst]
        return r_promote(op, f(a1))
    elseif ifirst + blksize > ilast
        # sequential portion
        @inbounds a1 = A[ifirst]
        @inbounds a2 = A[ifirst+1]
        v = op(r_promote(op, f(a1)), r_promote(op, f(a2)))
        @simd for i = ifirst + 2 : ilast
            @inbounds ai = A[i]
            v = op(v, f(ai))
        end
        return v
    else
        # pairwise portion
        imid = (ifirst + ilast) >> 1
        v1 = mapreduce_impl(f, op, A, ifirst, imid, blksize)
        v2 = mapreduce_impl(f, op, A, imid+1, ilast, blksize)
        return op(v1, v2)
    end
end
```

## Quicksort - less trivial example

In [ ]:
# stolen from https://stackoverflow.com/a/18262384
py"""
def pymyquicksort(array):
    less = []
    equal = []
    greater = []

    if len(array) > 1:
        pivot = array[0]
        for x in array:
            if x < pivot:
                less.append(x)
            if x == pivot:
                equal.append(x)
            if x > pivot:
                greater.append(x)
        # Don't forget to return something!
        return pymyquicksort(less) + equal + pymyquicksort(greater)  # Just use the + operator to join lists
    # Note that you want equal ^^^^^ not pivot
    else:  # You need to hande the part at the end of the recursion - when you only have one element in your array, just return the array.
        return array
"""
pymyquicksort = py"pymyquicksort"

In [ ]:
A = rand(100_000)
@time pymyquicksort(A)

In [ ]:
function myquicksort(A)
    length(A) > 1 || return A
    
    less = similar(A, 0)
    equal = similar(A, 0)
    greater = similar(A, 0)

    pivot = A[1]
    for x in A
        x < pivot && push!(less, x)
        x == pivot && push!(equal, x)
        x > pivot && push!(greater, x)
    end
    
    [ myquicksort(less); equal; myquicksort(greater) ]
end

In [ ]:
myquicksort(A)
@time myquicksort(A)

In [ ]:
@code_native myquicksort(A)

In [ ]:
square_area(a) = a * a

In [ ]:
code_native(square_area, (Int32,))

In [ ]:
code_native(square_area, (Float64,))

# Fresnel propagation

`PyPlot` is already installed, but installing it is as simple as:

`julia> Pkg.add("PyPlot")`

Julia has a built-in package manager.

In [ ]:
using PyPlot

In [ ]:
"The rectangle function."
rect(x) = one(x) * (abs(x) < 0.5)

"A circular grating"
circle(x, y, m, L) = m * rect(sqrt(x^2 + y^2) / 2L)

In [ ]:
x = linspace(-1, 1)
plot(x, rect.(x), ".-")

In [ ]:
"frequencies for which the fft is calculated"
fftfreq(n) = ifftshift(-cld(n-1, 2) : fld(n-1, 2)) / n

"Transfer function for the Fresnel free propagation."
H(fx, fy, λ, z) = exp(-im * π * λ * z * (fx^2 + fy^2))

In [ ]:
δx = δy = 0.1e-6
# Make sure Nx is a power of 2 for performance reasons
Nx = Ny = nextprod([2], 4000)

# x is a row vector, y is a column vector
x = δx * (collect(1:Nx) - Nx/2)'
y = δy * (collect(1:Ny) - Ny/2)

fx = (fftfreq(length(x)) ./ δx)'
fy = fftfreq(length(y)) ./ δy

In [ ]:
u0 = ones(Complex128, length(y), length(x))

# grating 20μm
p1 = 20e-6
u1 = u0 .* circle.(x, y, 1, p1)
U1 = fft(u1)

# wavelength 100pm
λ = 100e-12
u2(z) = ifft(U1 .* H.(fx, fy, λ, z), (1, 2))

"The distance for the given Fresnel number"
NF2z(NF) = (p1/2)^2 / (λ * NF)

In [ ]:
z = NF2z(2)
intensity_image = abs.(u2(z)).^2

In [ ]:
imshow(intensity_image,
    cmap = "Greys_r",
    interpolation = "bicubic",
    aspect = 1)
xlim(1800, 2300)
ylim(1800, 2300)

In [ ]:
@profile intensity_image = abs.(u2(z)).^2

In [ ]:
Profile.print()

# Intergrating with Python

In [ ]:
using PyCall

In [ ]:
@pyimport scipy.optimize as so
so.newton(x -> cos(x) - x, 1)

Single line `py"..."` returns a value, but has to be a single expression.

In [ ]:
py"len([12, 23])"

Multiline `py"""..."""` can execute arbitrary python code, but returns `nothing`.

In [ ]:
py"""
def python_add(a, b):
    return a + b
"""
python_add = py"python_add"

In [ ]:
python_add(2, 2)

In [ ]:
py"""
class PythonClass:
    pass

pyobject = PythonClass()
pyobject.field = "value"
"""
pyobject = py"pyobject"

In [ ]:
py"pyobject.field"

In [ ]:
pyobject[:field]

# Plotting with matplotlib

In [ ]:
using PyPlot

In [ ]:
errorbar(0:9, rand(10), fmt = ".", yerr = 0.1)
xlabel("whatever (a.u.)")
ylabel("intensity (a.u.)")

# Calling C

In [ ]:
ccall((:clock, "libc"), Int32, ())

In [ ]:
ccall((:getenv, "libc"), Cstring, (Cstring,), "SHELL") |> unsafe_string

Can easily wrap functions:

In [ ]:
getenv(v) = ccall((:getenv, "libc"), Cstring, (Cstring,), v) |> unsafe_string

In [ ]:
getenv("HOME")

In [ ]:
a = collect(1:10)

In [ ]:
pointer(a)

# C++ integration

In [ ]:
using Cxx

In [ ]:
cxx""" #include<iostream> """

In [ ]:
cxx"""  
  void mycppfunction() {   
    int z = 0;
    int y = 4;
    int x = 10;
    z = x*y + 2;
    std::cout << "The number is " << z << std::endl;
 }
"""

In [ ]:
julia_function() = @cxx mycppfunction()

In [ ]:
julia_function()

# Parallel computing

In [ ]:
incircle(x, y, r) = x^2 + y^2 < r^2

In [ ]:
function estimateπ(N)
    inside = 0
    for i in 1:N
        x = rand() - 0.5
        y = rand() - 0.5

        incircle(x, y, 0.5) && (inside += 1)
    end
    inside / N / 0.5^2
end
    

estimateπ(1000)
@time estimateπ(100_000_000)

In [ ]:
addprocs(4)

In [ ]:
@everywhere @show myid()

In [ ]:
@everywhere incircle(x, y, r) = x^2 + y^2 < r^2

In [ ]:
N = 100_000_000
Nworker = N ÷ nworkers()

@time inside = @parallel (+) for i in 1:N
    x = rand() - 0.5
    y = rand() - 0.5

    incircle(x, y, 0.5)
end

inside / N / 0.5^2

# Shell-capabilities

In [ ]:
run(`ls`)

In [ ]:
url = "https://raw.githubusercontent.com/rawlik/Coils.jl/master/Coils.jl"

In [ ]:
# get all the functions defined in that file
byline_matches = [ match(r"^function ([a-zA-Z0-9_]+)\(.*\)", s) for s in split(readstring(`curl $url`), "\n") ]
[ m.captures[] for m in byline_matches if m != nothing]

In [ ]:
# How many lines does the file have?
run(pipeline(`curl $url`, `wc -l`))

# Type system and multiple-dispatch

In [ ]:
methods(+)

# Metaprogramming

In [ ]:
x1 = 1

In [ ]:
x2 = 2

In [ ]:
typeof(:x1)

In [ ]:
eval(:x1)

In [ ]:
dump(:(sin(t)^2 + cos(p)^2))

In [ ]:
whichx = 1

In [ ]:
eval(parse("x$whichx"))

In [ ]:
for whichx in 1:2
    @show "x$whichx" |> parse |> eval
end

In [ ]:
# stolen from https://en.wikibooks.org/wiki/Introducing_Julia

macro until(condition, block)
    quote
        while true
            $(esc(block))
            if $(esc(condition))
                break
            end
        end
    end
end

In [ ]:
i = 0 
@until i == 10 begin
    i += 1
    @show i
end

# The down side of julia

In [ ]:
@time `julia -e "println(2+2)"` |> run

In [ ]:
@time `python3 -c "print(2+2)"` |> run

In [ ]:
@time `julia -e "using PyPlot"` |> run

In [ ]:
@time `python3 -c "from pylab import *"` |> run

# Crash-course

Based on `https://learnxinyminutes.com/docs/julia/` with modifications.

In [ ]:
2

In [ ]:
2 + 1im

In [ ]:
1im^2

In [ ]:
2//3

In [ ]:
pi

In [ ]:
2π

In [ ]:
2(3+2)

In [ ]:
5 / 2

In [ ]:
5 ÷ 2

In [ ]:
1 / 0

In [ ]:
2 << 1

Constants have constant *type*.

In [ ]:
const constant = 3

In [ ]:
constant = 4

In [ ]:
constant = 3.14

## Arrays

In [ ]:
a = [4, 5, 6]

In [ ]:
a[1]

In [ ]:
a = [4; 5; 6]

In [ ]:
a[end]

In [ ]:
a[end - 1]

In [ ]:
a[]

In [ ]:
a[[1, 3]]

In [ ]:
a[[true, false, true]]

In [ ]:
1:5

In [ ]:
collect(1:5)

In [ ]:
a[1:3]

In [ ]:
a[2:end]

In [ ]:
a[end:-1:1]

In [ ]:
a[1] = 0

In [ ]:
a

In [ ]:
matrix = [1 2; 3 4]

In [ ]:
matrix = [1 2
          3 4]

In [ ]:
b = Int8[4, 5, 6]

In [ ]:
10a

In [ ]:
matrix * [10, 20]

In [ ]:
# Julia distinguishes row and column vectors:
x = [1, 2]

In [ ]:
y = [1 2]

In [ ]:
x * y

In [ ]:
y * x

In [ ]:
matrix'

In [ ]:
[1, 2] * [3, 4]

In [ ]:
# Use the so-called dot operators to perform operations element-wise
[1, 2] .* [3, 4]

In [ ]:
# Dot operators support broadcasting - singleton dimensions are automatically expanded.
[1 2; 3 4] .* [10, 20]

In [ ]:
[1 2; 3 4] .* [10 10; 20 20]

In [ ]:
a

In [ ]:
a .= 7

In [ ]:
b = [1, 2, 3]
c = [10, 20, 30]

In [ ]:
a .= b .* c

In [ ]:
@. a = b * c

In [ ]:
a[a.>20]

## Strings

In [ ]:
s = "This is a string."

In [ ]:
"""You can
have multiline strings, too."""

In [ ]:
'a'

In [ ]:
s[1]

In [ ]:
s[1:4]

In [ ]:
utf8string = "Θεσσαλονίκη"

In [ ]:
utf8string[1]

In [ ]:
utf8string[2]

In [ ]:
[ c for c in utf8string ]

In [ ]:
s[1] = "T"

In [ ]:
# `$` is used for interpolation.
"pi is $pi"

In [ ]:
"2 + 2 = $(2 + 2)"

In [ ]:
@printf "%d is less than %f" 4.5 5.3

In [ ]:
@sprintf "%d is less than %f" 4.5 5.3

In [ ]:
print("Print without newline... ")
println("Print with newline at the end.")

In [ ]:
"one" * "two"

In [ ]:
"one"^3

In [ ]:
ismatch(r"^\s*(?:#|$)", "not a comment")

In [ ]:
ismatch(r"^\s*(?:#|$)", "# a comment")

In [ ]:
b"abc"

## Collections

In [ ]:
a = []
# Any is the supertype of all types  (more on that later).
# This means, that this vector can hold anything.

In [ ]:
# Add stuff to the end of a with push! (single element)
# and append! (another collection)
push!(a, 1)

In [ ]:
push!(a, 2)

In [ ]:
push!(a, 4)

In [ ]:
push!(a, 3)

In [ ]:
# By convention, function names that end in exclamations points indicate that
# they modify their arguments. Usually it is the first argument being modified.
append!(a, ["six", π])

In [ ]:
# Remove from the end with pop
pop!(a) # => π = 3.14…
a

In [ ]:
# we also have shift and unshift
shift!(a)

In [ ]:
a

In [ ]:
unshift!(a, 7)

In [ ]:
# Remove elements from an array by index with splice!
splice!(a, 2)

In [ ]:
a

In [ ]:
# Check for existence in a list with in
in(1, a)

In [ ]:
3 in a

In [ ]:
# Examine the length with length
length(a)

In [ ]:
# Sorting
arr = [5,4,6]

In [ ]:
sort(arr)

In [ ]:
arr

In [ ]:
sort!(arr)

In [ ]:
arr

In [ ]:
# Tuples are immutable.
tup = (1, 2, 3)

In [ ]:
tup[1] = 3

In [ ]:
# Many list functions also work on tuples
length(tup)

In [ ]:
tup[1:2]

In [ ]:
2 in tup

In [ ]:
# You can unpack tuples into variables
a, b, c = (1, 2, 3)

In [ ]:
a

In [ ]:
b

In [ ]:
c

In [ ]:
# Tuples are created even if you leave out the parentheses
d, g, h = 4, 5, 6

In [ ]:
# A 1-element tuple is distinct from the value it contains
(1,) == 1

In [ ]:
(1) == 1

In [ ]:
# Look how easy it is to swap two values
g, d = d, g

In [ ]:
g

In [ ]:
d

In [ ]:
# Dictionaries store mappings
empty_dict = Dict()

In [ ]:
# You can create a dictionary using a literal
filled_dict = Dict("one"=> 1, "two"=> 2, "three"=> 3)

In [ ]:
# Look up values with []
filled_dict["one"]

In [ ]:
# Get all keys
# Note - dictionary keys are not sorted or in the order you inserted them.
keys(filled_dict)

In [ ]:
# Get all values
values(filled_dict)

In [ ]:
# Check for existence of keys in a dictionary with in, haskey
in(("one" => 1), filled_dict)

In [ ]:
("two" => 3) in filled_dict

In [ ]:
haskey(filled_dict, "one")

In [ ]:
haskey(filled_dict, 1)

In [ ]:
filled_dict["four"]

In [ ]:
# Use the get method to avoid that error by providing a default value
# get(dictionary,key,default_value)
get(filled_dict, "one", 4)

In [ ]:
get(filled_dict, "four", 4)

In [ ]:
# Use Sets to represent collections of unordered, unique values
empty_set = Set()

In [ ]:
# Initialize a set with values
filled_set = Set([1,2,2,3,4])

In [ ]:
# Add more values to a set
push!(filled_set, 5)

In [ ]:
filled_set

In [ ]:
# Check if the values are in the set
2 in filled_set

In [ ]:
10 in filled_set

In [ ]:
# There are functions for set intersection, union, and difference.
other_set = Set([3, 4, 5, 6])

In [ ]:
intersect(filled_set, other_set)

In [ ]:
filled_set ∩ other_set # The same with ∩ (\cap)

In [ ]:
union(filled_set, other_set)

In [ ]:
filled_set ∪ other_set # The same ∪ (\cup)

In [ ]:
setdiff(Set([1,2,3,4]), Set([2,3,5]))

In [ ]:
# Set operations work on arrays, too
[1,2,3,4] ∩ [2,3,4]

## Control Flow

In [ ]:
# Let's make a variable
some_var = 5

In [ ]:
# Here is an if statement. Indentation is not meaningful in Julia.
if some_var > 10
    println("some_var is totally bigger than 10.")
elseif some_var < 10    # This elseif clause is optional.
    println("some_var is smaller than 10.")
else                    # The else clause is optional too.
    println("some_var is indeed 10.")
end

In [ ]:
# if statement can return variables
result = if some_var > 10
    "greater than 10"
else
    "smaller of equal 10"
end
result

In [ ]:
# There is the ternary operator
result = some_var > 5 ? "greater than 5" : "smaller of equal 5"
result

In [ ]:
# For loops iterate over iterables.
# Iterable types include Range, Array, Set, Dict, and AbstractString.
for animal = ["dog", "cat", "mouse"]
    println("$animal is a mammal")
end

In [ ]:
# You can use 'in' instead of '='.
for i in 1:3
    println("i is now $i")
end

In [ ]:
for a in Dict("dog" => "mammal", "cat" => "mammal", "mouse" => "mammal")
    println("$(a[1]) is a $(a[2])")
end

In [ ]:
for (k,v) in Dict("dog" => "mammal", "cat" => "mammal", "mouse" => "mammal")
    println("$k is a $v")
end

In [ ]:
# While loops loop while a condition is true
x = 0
while x < 4
    println(x)
    x += 1  # Shorthand for x = x + 1
end

In [ ]:
# Handle exceptions with a try/catch block
try
   error("help")
catch e
   println("caught it $e")
end

## Functions

In [ ]:
# The keyword 'function' creates new functions
# function name(arglist)
#   body...
# end
function add(x, y)
    println("x is $x and y is $y")

    # Functions return the value of their last statement
    x + y
end

In [ ]:
add(5, 6)

In [ ]:
# By the way, every binary operator is a function, too, so:
+(5, 6)

In [ ]:
# Compact assignment of functions
add(x, y) = x + y
add(3, 4)

In [ ]:
# Function can also return multiple values as tuple
add_subtract(x, y) = x + y, x - y
add_subtract(3, 4)

In [ ]:
# You can define functions that take a variable number of
# positional arguments
function varargs(args...)
    return args
    # use the keyword return to return anywhere in the function
end

v

In [ ]:
# You can define functions that take a variable number of
# positional arguments
function varargs(args...)
    return args
    # use the keyword return to return anywhere in the function
end
varargs(1,2,3)

In [ ]:
# The ... is called a splat.
# We just used it in a function definition.
# It can also be used in a function call,
# where it will splat an Array or Tuple's contents into the argument list.
add([5,6]...) # this is equivalent to add(5,6)

In [ ]:
x = (5,6)     # => (5,6)
add(x...)     # this is equivalent to add(5,6)

In [ ]:
# You can define functions with optional positional arguments
function defaults(a, b, x = 5, y = 6)
    return "$a $b and $x $y"
end

defaults('h','g')

In [ ]:
defaults('h','g','j')

In [ ]:
defaults('h','g','j','k')

In [ ]:
defaults('h')

In [ ]:
defaults()

In [ ]:
# You can define functions that take keyword arguments,
# specified after a semicolon
function keyword_args(; k1 = 4, name2 = "hello") # note the ;
    return Dict("k1" => k1, "name2" => name2)
end

keyword_args(name2 = "ness")

In [ ]:
keyword_args(k1 = "mine")

In [ ]:
keyword_args()

In [ ]:
# You can combine all kinds of arguments in the same function
function all_the_args(normal_arg, optional_positional_arg=2; keyword_arg="foo")
    println("normal arg: $normal_arg")
    println("optional arg: $optional_positional_arg")
    println("keyword arg: $keyword_arg")
end

all_the_args(1, 3, keyword_arg=4)

In [ ]:
# Julia has first class functions
function create_adder(x)
    adder = function (y)
        return x + y
    end
    return adder
end

In [ ]:
# This is "stabby lambda syntax" for creating anonymous functions
(x -> x > 2)(3)

In [ ]:
# This function is identical to create_adder implementation above.
function create_adder(x)
    y -> x + y
end

In [ ]:
# You can also name the internal function, if you want
function create_adder(x)
    function adder(y)
        x + y
    end
    adder
end

add_10 = create_adder(10)
add_10(3)

In [ ]:
# There are built-in higher order functions
map(iszero, [1, 0, 3, 0])

In [ ]:
# negating a function gives a function with its result negated
map(!iszero, [1, 0, 3, 0])

In [ ]:
filter(x -> x > 5, [3, 4, 5, 6, 7])

In [ ]:
# The "do" block creates an anonymous function and passes is as the first
# argument.
result = map([1, 2, 3, 4, 5, 6]) do x
    x % 2 == 0
end
result

In [ ]:
result == map(x -> x % 2 == 0, [1, 2, 3, 4, 5, 6])

In [ ]:
# The binary operator ∘ (\circ) combines functions
map( (-) ∘ √, [1, 4, 16])

In [ ]:
# √, \sqrt[TAB], is the square root. It can also be used without parentheses
√2 # => 1.41…

In [ ]:
# We can use list comprehensions for nicer maps
[add_10(i) for i = [1, 2, 3]]

In [ ]:
[add_10(i) for i in [1, 2, 3]]

In [ ]:
# List comprehensions work in more dimensions, too
[x + y for x in 1:2, y in 10:10:30]

In [ ]:
# Function followed by a dot acts element-wise
isfinite.([1, 1/0, NaN]) # => [true, false, false]

In [ ]:
x = rand(3)
abs.(sin.(x).^2 .+ cos.(x).^2 .- 1) .≤ eps(Float64) # => [true, true, true]
# Many dot operations are automatically fused into a single loop.

In [ ]:
# @. adds a dot to every function and operator after it
@. abs(sin(x)^2 + cos(x)^2 - 1) ≤ eps(Float64) # => [true, true, true]

## Types

In [ ]:
# Julia has a type system.
# Every value has a type; variables do not have types themselves.
# You can use the `typeof` function to get the type of a value.
typeof(5) # => Int64

In [ ]:
# Types are first-class values
typeof(Int64)

In [ ]:
typeof(DataType)
# DataType is the type that represents types, including itself.

In [ ]:
# Types are used for documentation, optimizations, and dispatch.
# They are not statically checked.

# Users can define types
struct Tiger
  taillength::Float64
  coatcolor # not including a type annotation is the same as `::Any`
end

In [ ]:
# The default constructor's arguments are the properties
# of the type, in the order they are listed in the definition
tigger = Tiger(3.5, "orange")

In [ ]:
# The type doubles as the constructor function for values of that type
sherekhan = typeof(tigger)(5.6, "fire")

In [ ]:
# These struct-style types are called concrete types
# They can be instantiated, but cannot have subtypes.
# The other kind of types is abstract types.

# abstract type Name
abstract type Cat  # just a name and point in the type hierarchy
end

In [ ]:
# Abstract types cannot be instantiated, but can have subtypes.
# For example, Number is an abstract type
subtypes(Number)

In [ ]:
subtypes(Cat)

In [ ]:
# AbstractString, as the name implies, is also an abstract type
subtypes(AbstractString)

In [ ]:
# Every type has a super type; use the `supertype` function to get it.
typeof(5)

In [ ]:
supertype(Int64)

In [ ]:
supertype(Signed)

In [ ]:
supertype(Integer)

In [ ]:
supertype(Real)

In [ ]:
supertype(Number)

In [ ]:
supertype(Any) # => Any

In [ ]:
# All of these type, except for Int64, are abstract.
typeof("fire")

In [ ]:
supertype(String)

In [ ]:
supertype(AbstractString)

In [ ]:
# <: is the subtyping operator
type Lion <: Cat # Lion is a subtype of Cat
  mane_color
  roar::AbstractString
end

In [ ]:
# You can define more constructors for your type
# Just define a function of the same name as the type
# and call an existing constructor to get a value of the correct type
Lion(roar::AbstractString) = Lion("green", roar)
# This is an outer constructor because it's outside the type definition

In [ ]:
type Panther <: Cat # Panther is also a subtype of Cat
  eye_color
  Panther() = new("green")
  # Panthers will only have this constructor, and no default constructor.
  # new() is a spetial function available in inner constructors, that creates
  # objects of the block's type.
end
# Using inner constructors, like Panther does, gives you control
# over how values of the type can be created.
# When possible, you should use outer constructors rather than inner ones.

## Multiple dispatch

In [ ]:
# In Julia, all named functions are generic functions
# This means that they are built up from many small methods
# Each constructor for Lion is a method of the generic function Lion.

# For a non-constructor example, let's make a function meow:

# Definitions for Lion, Panther, Tiger
function meow(animal::Lion)
    animal.roar # access type properties using dot notation
end

function meow(animal::Panther)
    "grrr"
end

function meow(animal::Tiger)
    "rawwwr"
end

In [ ]:
# Testing the meow function
meow(tigger)

In [ ]:
meow(Lion("brown", "ROAAR"))

In [ ]:
meow(Panther())

In [ ]:
# Review the local type hierarchy
issubtype(Tiger, Cat)

In [ ]:
issubtype(Lion, Cat)

In [ ]:
issubtype(Panther, Cat)

In [ ]:
# Defining a function that takes Cats
function pet_cat(cat::Cat)
    println("The cat says $(meow(cat))")
end

pet_cat(Lion("42"))

In [ ]:
pet_cat(tigger)

In [ ]:
# In OO languages, single dispatch is common;
# this means that the method is picked based on the type of the first argument.
# In Julia, all of the argument types contribute to selecting the best method.

# Let's define a function with more arguments, so we can see the difference
function fight(t::Tiger, c::Cat)
    println("The $(t.coatcolor) tiger wins!")
end

In [ ]:
fight(tigger, Panther())

In [ ]:
fight(tigger, Lion("ROAR"))

In [ ]:
# Let's change the behavior when the Cat is specifically a Lion
fight(t::Tiger, l::Lion) = println("The $(l.mane_color)-maned lion wins!")

In [ ]:
fight(tigger, Panther())

In [ ]:
fight(tigger, Lion("ROAR"))

In [ ]:
# We don't need a Tiger in order to fight
fight(l::Lion, c::Cat) = println("The victorious cat says $(meow(c))")
fight(Lion("balooga!"), Panther())

In [ ]:
# review the methods we created
methods(fight)

In [ ]:
fight(Panther(), Lion("RAWR"))

In [ ]:
# Also let the cat go first
fight(c::Cat, l::Lion) = println("The cat beats the Lion")

In [ ]:
fight(Lion("RAR"), Lion("brown", "rarrr"))

In [ ]:
fight(l1::Lion, l2::Lion) = println("The lions come to a tie")
fight(Lion("RAR"), Lion("brown", "rarrr"))

## Macros

In [ ]:
v = 9
@show 10 * √v

In [ ]:
@time inv(rand(100, 100))

In [ ]:
@time inv(rand(100, 100))
# prints "    0.001164 seconds (19 allocations: 286.781 KiB)"
# The second time is faster. The first time the functions were compiled.

In [ ]:
a = rand(10)
pointer(a)

In [ ]:
# Slicing produces a new Array
pointer(a[1:10]) == pointer(a)

In [ ]:
# The @view macro produces a SubArray sharing memory with the original
pointer(@view a[1:10]) == pointer(a)